<a href="https://colab.research.google.com/github/AliceKitchkin/Forecasting-Rossmann-Store-Sales/blob/main/Forecasting_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorhersagen von Rossmann Store Sales

## 1. Deskriptive Analyse

## 2. Geeignete Merkmale

## 3. Modellgüte

## 4. Machine Learning Verfahren 1

## 5. Machine Learning Verfahren 2

## 6. Vergleich der Machine Learning Verfahren